In [1]:
import sys
sys.path.insert(0, '/home/madan/Desktop/joswin_bck/toPendrive/works/analytics_work_bench/nlp-intelligence/analytics_workbench')

In [2]:
import pandas as pd
from analytics_workbench.read_data import DataReader
from analytics_workbench.preprocessing import PreProcessor
from analytics_workbench.process_text import ProcessText
from analytics_workbench.unsupervised_learning import Unsupervised
from analytics_workbench.supervised_learning import ClassificationModelCV
from analytics_workbench.model_diagnostics import ClassificationModelDiagnotstics
from sklearn import metrics
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss,f1_score
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC,SVC
from sklearn.multiclass import OneVsRestClassifier
import dill
import pickle
import cPickle

In [3]:
import psycopg2
from bs4 import BeautifulSoup
import re

In [4]:
con = psycopg2.connect(database='crawler_service_test', user='postgres',password='postgres',host='localhost')

In [5]:
cursor = con.cursor()

In [12]:
query = "select * from (select distinct on (a.domain) a.domain,categories,home_page_source,misc_details "\
                       "from webpage_texts_bck a join ecommerce_classifcation_alexa b using(domain))a where "\
                       "home_page_source is not null limit 1"

In [9]:
cursor.execute(query)

In [10]:
res = cursor.fetchall()

In [14]:
domain,category,page_source,dets = res[0]

In [17]:
soup = BeautifulSoup(page_source,'html.parser')

In [21]:
' '.join([i.get('content','')
     for i in soup.find_all('meta')
        if re.search('description|keyword',i.get('name','none'),re.IGNORECASE) or 
         re.search('description|keyword',i.get('property','none'),re.IGNORECASE) ])

u'11x17 Office Supplies for Engineering Drawings. 11 x 17 binder, 11 x 17 binders, 11 x 17 clipboards, 11 x 17 clipboard, 11 x 17 box, 11 x 17 cardstock, 11 x 17 folders, 11 x 17 file folders, 11 x 17 portfolio, 11 x 17 storage boxes'

In [26]:
tags = ['h1','h2','h3','h4','h5','h6','p']
tag_texts = []
for tag in tags:
    tag_texts.extend([i.text for i in soup.find_all(tag)])
print '. '.join(tag_texts)

Your Source For Big Office Solutions. Featured Best Sellers. Follow Us!. Catalog Request.   Free Standard Shipping for order of $75.00 or more. see details. Questions? Call Us:. (903) 541.0100. We take pride in being the best source for your office needs! Give us a call and our sales team will be glad to assist you with our office solutions. . (No reviews). Be the first to. (No reviews). Be the first to. (No reviews). Be the first to. (14 reviews). (5 reviews). (3 reviews). Out of stock. Need a Business Credit Application APPLY FOR CREDIT TERMS.  . Hello!. Select your country


In [35]:
' '.join([i.text.strip() for i in soup.find_all('a') if i.text.strip()])

u'Home Contact About Login or Register My Account see details Binders 8.5 x 11Letter Size 297mm x 210mmA4 8.5 x 14Legal 11 x 17Tabloid/Ledger 297mm x 420mmA3 13 x 19Super-B 18 x 24Arch-C Shop All Binders Accessories Fasteners Identification Pockets Index Tab Dividers Labels Lamination Pouches Presentation Covers Sheet Lifters Sheet Protectors Shop All Accessories Clipboards 8.5 x 14Legal 11 x 17Tabloid/Ledger 13 x 19Super-B 18 x 24Arch-C Shop All Clipboards Filing Systems Desk Trays Folders Portfolio Cases Storage Systems Wall Pockets Shop All Filing Systems Paper 8.5 x 11Letter Size 297mm x 210mmA4 8.5 x 14Legal 11 x 17Tabloid/Ledger 297mm x 420mmA3 13 x 19Super-B Shop All Paper Custom Printing Banners Report Covers T-Shirts Hats & Caps Binders Shop All Custom Printing 1 2 3 Previous Next 11x17 Binder - 1" Angle-D Ring Vinyl with Outside Pockets Write a Review 11x17 Binder - 2" Angle-D Ring Vinyl with Outside Pockets Write a Review 11x17 Index Tab Divider, 8 Tab, 1 Sets (Extra Long wi

In [6]:
#combining all above
query = "select * from (select distinct on (a.domain) a.domain,categories,home_page_source,misc_details "\
                       "from webpage_texts_bck a join ecommerce_classifcation_alexa b using(domain))a where "\
                       "home_page_source is not null"

In [7]:
cursor.execute(query)

In [8]:
res = cursor.fetchone()

In [9]:
domains_all,cats_all,meta_texts_all,page_texts_all,url_texts_all = [],[],[],[],[]

In [10]:
tags = ['h1','h2','h3','h4','h5','h6','p']
while res:
    domain,category,page_source,dets = res
    if not re.search('Check the address for typing errors such as',page_source):
        soup = BeautifulSoup(page_source,'html.parser')
        meta_text = ' '.join([i.get('content','')
             for i in soup.find_all('meta')
                if re.search('description|keyword',i.get('name','none'),re.IGNORECASE) or 
                 re.search('description|keyword',i.get('property','none'),re.IGNORECASE) ])
        tag_text = []
        for tag in tags:
            tag_text.extend([i.text.strip() for i in soup.find_all(tag) if i.text])
        page_text = ' '.join(tag_text)
        url_text = ' '.join([i.text.strip() for i in soup.find_all('a') if i.text])
        domains_all.append(domain)
        cats_all.append(category)
        meta_texts_all.append(meta_text)
        page_texts_all.append(page_text)
        url_texts_all.append(url_text)
    res = cursor.fetchone()

In [11]:
con.close()

In [12]:
df = pd.DataFrame({'domain':domains_all,'category':cats_all,'meta_text':meta_texts_all,'page_text':page_texts_all,
                   'url_text':url_texts_all})

In [13]:
df.head(50)

,category,domain,meta_text,page_text,url_text
0,Office_Products,11x17.com,11x17 Office Supplies for Engineering Drawings...,Your Source For Big Office Solutions Featured ...,Home Contact About Login or Register My Accoun...
1,Consumer_Electronics,a1components.com,"a1Components, audio video, tools, cables, spea...",Welcome to a1Components! Special Offers Popula...,Login My Account Help View Cart\n (0)\n...
2,Sporting_Goods,a1darts.com,,Error. Page cannot be displayed. Please contac...,
3,Pets,a1-dogs.com,"dog products,dog infographics, dog supplies, d...",Like \n our site?\nHere's \n ...,HOME DOG PRODUCTS Â Here's \n how...
4,Flowers,a1florists.com,"fresh flowers, potted plants, fruit baskets, d...",Simply\nchoose a florist or location on the fo...,Cities and Towns Business Cards Home\nPage Ci...
5,Health_and_Healthcare,a1-medical-supplies.net,"Lift chairs, stair lifts, vertical lifts, and ...","Factory Direct Lift Chairs,Stair Lifts,Wheelch...",A1 Coupons View Cart A1 Medical Home A1 Sales ...
6,Pets,a1pythons.com,"Henry Piorun, Piorun, snakes, Ball Python, pyt...",Welcome to Henry Piorun Reptiles and our web...,Assorted Carpet Pythons Ball Pythons 2014 - R...
7,Clothing,a1scrubs.com,A1Scrubs.com Nursing scrubs and accessories. n...,"Shop by brand to ensure the best fit, feel and...",Home Login Register become a VIP 866-217-2782 ...
8,Health_and_Healthcare,a1supplements.com,,AllMax Nutrition AllWhey Classic Pure Whey-Pr...,Protein Whey Protein Powder Beef Protein Casei...
9,Computers,a1toners.com,A1toners.com features a large selection of ton...,,Shopping Cart 0 Items in Cart Subtotal: $0.00 ...


In [25]:
df.to_csv('sample_ecom_data_feb21.csv',index=False,quoting=1,encoding='utf-8')

In [4]:
df = pd.read_csv('sample_ecom_data_feb21.csv')
df = df.fillna('')

In [5]:
df.loc[df['category'].isin(['Holidays_and_Celebrations','Beauty_Products','Crafts']),'category'] = 'other_categories_1'
df.loc[df['category'].isin(['Auctions', 'Publications', 'Computers', 'Consumer_Information', 'Death_Care', 'Health',
                          'Travel_Items', 'Sports', 'Toys_and_Games', 'Ethnic_and_Regional', 'Tools', 'Niche', 
                          'Weddings', 'Holidays', 'Tobacco']),'category'] = 'other_categories_2'
df['category'].value_counts()

Home_and_Garden              1523
Clothing                     1084
other_categories_2           1024
Sporting_Goods               1008
Food                          762
Recreation                    671
Antiques_and_Collectibles     633
Consumer_Electronics          609
Pets                          483
Jewelry                       481
Music                         424
Entertainment                 397
Health_and_Healthcare         367
Books                         350
General_Merchandise           341
Gifts                         324
Children                      318
other_categories_1            305
Flowers                       256
Other                         253
Office_Products               247
Photography                   213
Classifieds                   162
Name: category, dtype: int64

In [6]:
text_processor = ProcessText()
tfidf,vocabulary = text_processor.gen_dtm_from_files(df[['meta_text','page_text','url_text']].apply(lambda x: '. '.join(list(x)),1),
                                                     vectorizer_type='Count',stem_type=None,max_df=0.9,min_df=0.01,
                    n_gram_range=(1,2),stop_words_loc='/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/e-commerce_classification/alexa_based/stopwords.txt')

In [7]:
tfidf.shape

(12235, 3524)

In [9]:
with open('count_vector_feb23.pkl','w') as f:
    pickle.dump({'tf':tfidf,'vocabulary':vocabulary,'dv':df['category']},f)

In [4]:
with open('count_vector_feb23.pkl','r') as f:
    val_dic = pickle.load(f)
tfidf,vocabulary,dv = val_dic['tf'],val_dic['vocabulary'],val_dic['dv']
del val_dic

In [5]:
from sklearn import metrics
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss,f1_score
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC,SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV

In [6]:
lbl = preprocessing.LabelEncoder()
categories_transformed = lbl.fit_transform(dv.values)

In [7]:
predictions = np.zeros(categories_transformed.shape)

cv_f1_scores,cv_log_loss = [],[]
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(tfidf.shape[0])):
        dev_X, val_X = tfidf[dev_index,:], tfidf[val_index,:]
        dev_y, val_y = categories_transformed[dev_index], categories_transformed[val_index]
        clf = MultinomialNB(fit_prior=False)
        clf.fit(dev_X,dev_y)
        preds_labels = clf.predict(val_X)
        predictions[val_index] = preds_labels
        cv_f1_scores.append(f1_score(val_y, preds_labels,average='weighted'))
        #cv_log_loss.append(log_loss(val_y,preds_labels))
        print(cv_f1_scores)
        #print(cv_log_loss)

[0.61283728037711827]
[0.61283728037711827, 0.60524068383963803]
[0.61283728037711827, 0.60524068383963803, 0.58968947755710355]
[0.61283728037711827, 0.60524068383963803, 0.58968947755710355, 0.62086539796923901]
[0.61283728037711827, 0.60524068383963803, 0.58968947755710355, 0.62086539796923901, 0.61402396604197729]


In [8]:
print metrics.classification_report(categories_transformed,predictions,labels=range(23),target_names=list(lbl.classes_))
pd.DataFrame(metrics.confusion_matrix(categories_transformed,predictions),columns=list(lbl.classes_),index=list(lbl.classes_))

                           precision    recall  f1-score   support

Antiques_and_Collectibles       0.34      0.67      0.45       633
                    Books       0.35      0.63      0.45       350
                 Children       0.61      0.64      0.62       318
              Classifieds       0.39      0.87      0.54       162
                 Clothing       0.74      0.61      0.67      1084
     Consumer_Electronics       0.68      0.62      0.65       609
            Entertainment       0.54      0.56      0.55       397
                  Flowers       0.84      0.88      0.86       256
                     Food       0.70      0.78      0.74       762
      General_Merchandise       0.27      0.21      0.24       341
                    Gifts       0.60      0.61      0.61       324
    Health_and_Healthcare       0.46      0.61      0.53       367
          Home_and_Garden       0.79      0.56      0.66      1523
                  Jewelry       0.86      0.72      0.78     

,Antiques_and_Collectibles,Books,Children,Classifieds,Clothing,Consumer_Electronics,Entertainment,Flowers,Food,General_Merchandise,...,Jewelry,Music,Office_Products,Other,Pets,Photography,Recreation,Sporting_Goods,other_categories_1,other_categories_2
Antiques_and_Collectibles,423,19,1,14,24,2,8,0,15,9,...,12,1,8,7,3,5,25,7,5,18
Books,29,220,5,11,0,3,9,0,12,2,...,1,6,2,1,5,1,7,3,2,13
Children,13,17,203,7,7,0,5,1,5,4,...,3,3,2,0,3,1,0,4,4,9
Classifieds,7,1,0,141,0,0,2,1,1,0,...,0,0,2,0,2,0,0,0,1,2
Clothing,61,31,27,23,664,6,19,0,26,17,...,6,2,12,29,8,2,11,23,12,73
Consumer_Electronics,48,14,1,12,4,379,9,0,11,7,...,0,8,3,0,2,14,5,1,0,67
Entertainment,28,23,1,8,6,7,222,2,5,1,...,0,23,4,3,2,2,3,8,5,26
Flowers,9,2,2,1,0,0,1,225,3,0,...,0,0,0,0,1,0,1,0,2,3
Food,39,15,0,12,4,2,4,9,593,6,...,0,2,0,1,7,0,4,2,2,18
General_Merchandise,26,23,6,4,21,7,7,2,11,73,...,10,1,3,12,5,1,7,2,13,63


In [22]:
preds = pd.DataFrame({'domain':df['domain'],'actual':df['category'],'prediction':lbl.inverse_transform(predictions.astype(int))})
preds[preds['actual']!=preds['prediction']]

,actual,domain,prediction
2,Sporting_Goods,a1darts.com,Photography
12,Sporting_Goods,a2bsports.com,Antiques_and_Collectibles
16,Entertainment,aaaanime.com,Antiques_and_Collectibles
18,Jewelry,aaajewelry.com,Classifieds
19,Health_and_Healthcare,aaamobility.com,Antiques_and_Collectibles
21,Jewelry,aaawatchclub.com,Antiques_and_Collectibles
23,Antiques_and_Collectibles,aabm.be,Food
24,Home_and_Garden,aabreecoffee.com,Food
25,Clothing,aabsinthcorsets.com,Health_and_Healthcare
26,other_categories_1,aacardi.com,Books


In [23]:
preds[preds['actual']==preds['prediction']]

,actual,domain,prediction
0,Office_Products,11x17.com,Office_Products
1,Consumer_Electronics,a1components.com,Consumer_Electronics
3,Pets,a1-dogs.com,Pets
4,Flowers,a1florists.com,Flowers
5,Health_and_Healthcare,a1-medical-supplies.net,Health_and_Healthcare
6,Pets,a1pythons.com,Pets
7,Clothing,a1scrubs.com,Clothing
8,Health_and_Healthcare,a1supplements.com,Health_and_Healthcare
9,other_categories_2,a1toners.com,other_categories_2
10,Clothing,a1uniforms.com,Clothing


In [28]:
#l2 penalty 0.58,l1 0.57
#cv l2 0.593 l1 0.5087 (??)
#l2 c 1 0.577 0.1 0.5963 0.01 0.6161,0.001 [0.60718448740041808]
predictions = np.zeros(categories_transformed.shape)

cv_f1_scores,cv_log_loss = [],[]
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(tfidf.shape[0])):
        dev_X, val_X = tfidf[dev_index,:], tfidf[val_index,:]
        dev_y, val_y = categories_transformed[dev_index], categories_transformed[val_index]
        clf = LogisticRegression(penalty='l2',C=0.001)
        clf.fit(dev_X,dev_y)
        preds_labels = clf.predict(val_X)
        predictions[val_index] = preds_labels
        cv_f1_scores.append(f1_score(val_y, preds_labels,average='weighted'))
        #cv_log_loss.append(log_loss(val_y,preds_labels))
        print(cv_f1_scores)
        break

[0.60718448740041808]


In [21]:
print metrics.classification_report(categories_transformed,predictions,labels=range(23),target_names=list(lbl.classes_))
pd.DataFrame(metrics.confusion_matrix(categories_transformed,predictions),columns=list(lbl.classes_),index=list(lbl.classes_))

                           precision    recall  f1-score   support

Antiques_and_Collectibles       0.61      0.51      0.56       633
                    Books       0.56      0.48      0.52       350
                 Children       0.62      0.48      0.55       318
              Classifieds       0.75      0.71      0.73       162
                 Clothing       0.55      0.62      0.58      1084
     Consumer_Electronics       0.66      0.55      0.60       609
            Entertainment       0.56      0.45      0.50       397
                  Flowers       0.90      0.82      0.86       256
                     Food       0.74      0.72      0.73       762
      General_Merchandise       0.27      0.19      0.22       341
                    Gifts       0.60      0.52      0.55       324
    Health_and_Healthcare       0.58      0.45      0.50       367
          Home_and_Garden       0.43      0.69      0.53      1523
                  Jewelry       0.80      0.71      0.75     

,Antiques_and_Collectibles,Books,Children,Classifieds,Clothing,Consumer_Electronics,Entertainment,Flowers,Food,General_Merchandise,...,Jewelry,Music,Office_Products,Other,Pets,Photography,Recreation,Sporting_Goods,other_categories_1,other_categories_2
Antiques_and_Collectibles,324,6,6,4,25,3,7,1,4,5,...,18,4,9,6,6,4,19,29,2,39
Books,17,168,5,6,13,6,5,1,6,2,...,2,4,0,6,3,3,16,9,1,31
Children,2,7,154,1,31,0,6,0,2,2,...,3,1,0,5,1,0,6,10,5,17
Classifieds,4,0,0,115,3,2,0,1,2,3,...,0,0,1,0,1,0,1,5,0,8
Clothing,12,4,16,0,667,8,9,0,12,17,...,5,2,5,14,7,1,15,59,8,65
Consumer_Electronics,6,7,1,2,29,337,8,1,2,7,...,0,8,3,4,2,11,11,17,2,53
Entertainment,8,11,4,3,19,9,179,1,5,4,...,3,27,1,3,6,1,12,12,6,26
Flowers,1,1,0,1,3,0,1,210,5,0,...,0,1,0,1,1,0,1,0,3,4
Food,6,2,0,1,22,2,3,4,547,10,...,1,0,0,5,8,0,6,11,2,16
General_Merchandise,10,6,7,2,32,5,4,2,12,65,...,11,3,2,9,8,1,16,12,7,42


#### topic extraction try

In [15]:
# check for clothing
unsup_obj = Unsupervised()
model,mat_transform = unsup_obj.generate_topics(tfidf[np.where(dv=='General_Merchandise')[0],:],vocabulary,'lda',50,10)

Topic #0:
art*56.51, ebay*14.36, san*8.04, lovers*6.76, motor*5.52, prep*4.68, weekend*4.67, tech*4.64, rate*4.63, western*4.26
Topic #1:
mobile*20.89, applications*5.86, codes*5.63, business*5.48, development*5.03, inventory*4.89, apps*3.61, management*2.58, application*2.42, code*2.35
Topic #2:
collectibles*1.74, width*1.73, media*1.62, music*1.57, image*1.34, plates*1.30, border*1.20, class*1.18, mail*1.15, style*1.12
Topic #3:
music*0.77, collectibles*0.66, license*0.62, company*0.61, add cart*0.60, plates*0.59, platform*0.59, lead*0.58, bottom*0.58, animal*0.56
Topic #4:
accessories*1.51, furniture*1.49, kitchen*1.41, pc*1.29, magnets*1.20, save*1.10, shop*1.08, cart*1.01, date*1.00, outdoor*0.98
Topic #5:
mobile*0.99, development*0.86, apps*0.61, ny*0.59, pool*0.58, reasonable*0.57, soon*0.57, business*0.57, service*0.57, codes*0.57
Topic #6:
amazon*0.85, company*0.84, profile*0.69, angel*0.65, projects*0.65, credit*0.63, companies*0.60, services*0.58, one*0.58, shipping*0.58
Top

In [9]:
unsup_obj = Unsupervised()
model,clusts,cust_dists = unsup_obj.generate_clusters(tfidf[np.where(dv=='Clothing')[0],:],vocabulary,10)

No of items in each cluster
8    905
1    153
0     11
2      8
9      2
7      1
6      1
5      1
4      1
3      1
dtype: int64
Top terms per cluster:
()
Cluster 0: shoes, wedding, dresses, bridal, add, border, cart, add cart, dress, color
Cluster 1: shop, shoes, new, women, accessories, hats, size, dresses, shirts, clothing
Cluster 2: boots, shirts, jeans, shoes, pants, accessories, women, dress, jackets, shorts
Cluster 3: logo, school, girls, xl, adult, logos, youth, half, size, boys
Cluster 4: socks, kids, novelty, ladies, knee, crew, athletic, high, kid, blend
Cluster 5: silk, ties, pocket, bow, pattern, solid, print, wool, scarves, new
Cluster 6: content, width, catalog, shop, data, max, sizes, size, height, class
Cluster 7: bridal, dresses, wedding, lace, accessories, style, options, jewelry, choose options, choose
Cluster 8: shop, new, contact, accessories, shirts, shoes, clothing, women, fashion, home
Cluster 9: size, plus, plus size, western, dresses, women, wear, clothing,